In [ ]:
%load_ext autoreload
%autoreload 2

## Digging deeper into convolutional networks

Lets check out the first level of filters in a high-performing neural network!

In [ ]:
from keras.applications.vgg16 import VGG16

neural_net = VGG16(include_top=False,
                      input_shape=(224,224,3),
                      pooling='avg')

In [ ]:
neural_net.summary()

In [ ]:
first_conv_weights = neural_net.get_layer('block1_conv1').get_weights()
for f in first_conv_weights:
    print(f.shape)

How many filters are there? What is their dimension?

In [ ]:
# can we treat filters as images themselves? Lets try!
from PIL import Image
import numpy as np

def filter2image(one_filter, resize=True):
    '''Makes visualizing filters possible -- very crudely with poor normalization'''
    one_filter += np.min(one_filter)
    one_filter /= np.max(one_filter)
    one_filter *= 255
    one_filter = one_filter.astype(np.uint8)
    one_filter = Image.fromarray(one_filter)
    if resize:
        one_filter = one_filter.resize((100,100))
    return one_filter

In [ ]:
# get access to the first filter and display it...
for filterc in range(64):
    one_filter = # Display the filter weights
    display(filter2image(one_filter))

perhaps more interesting is visualizing the later-layer activations for some images...

In [ ]:
from keras.models import Model
from util import load_images_for_neural_network
test_images = ['coco_workshop/22.jpg', 'coco_workshop/23.jpg', 'coco_workshop/24.jpg', 'coco_workshop/25.jpg']

images_for_nn = load_images_for_neural_network(test_images,
                                               batch_size=4)
extract_model = Model(inputs=neural_net.input,
                      outputs=neural_net.get_layer('block1_conv1').output)

In [ ]:
result = extract_model.predict_generator(images_for_nn,
                                         steps=1,
                                         verbose=1)

In [ ]:
print(result.shape)

Checkin: what do each of these dimensions correspond to?

In [ ]:
n_filters = 10
for idx, t in enumerate(test_images):
    my_image = Image.open(t)
    display(my_image)
    for filter_idx in range(n_filters):
        c_filter = result[idx,:,:,filter_idx]
        display(filter2image(c_filter, resize=False))

What about layers later-on in the processing pipeline?

In [ ]:
images_for_nn = load_images_for_neural_network(test_images,
                                               batch_size=4)
extract_model = Model(inputs=neural_net.input,
                      outputs=neural_net.get_layer('block3_conv3').output)
result = extract_model.predict_generator(images_for_nn,
                                         steps=1,
                                         verbose=1)

In [ ]:
print(result.shape)

Checkin: what do the dimensions represent?

In [ ]:
n_filters = 10
for idx, t in enumerate(test_images):
    my_image = Image.open(t)
    display(my_image)
    for filter_idx in range(n_filters):
        c_filter = result[idx,:,:,filter_idx]
        display(filter2image(c_filter, resize=True))